<a href="https://colab.research.google.com/github/sands-eg/GEE_training/blob/main/PLANET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificating PLANET data

In [2]:
import ee

In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=9HNF_mUOMYElN_LfY4-xuN1peabvHsLMVd6myIt6dhU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWja1mxY5MqKTQbzUkneQa85FeEHBiamgtuHkxj658R34zcb8OSRokc

Successfully saved authorization token.


In [17]:
import pprint
import folium
!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro

In [10]:
# loading an image 
PL2021 = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia").filterBounds(ee.Geometry.Rectangle([[117.586,0.668],[117.756,0.805]])).filterDate('2021-01-01', '2021-12-31').first()
# loading an image 
PL2020 = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia").filterBounds(ee.Geometry.Rectangle([[117.586,0.668],[117.756,0.805]])).filterDate('2020-01-01', '2020-12-31').first()
# loading an image 
PL2019 = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia").filterBounds(ee.Geometry.Rectangle([[117.586,0.668],[117.756,0.805]])).filterDate('2019-01-01', '2019-12-31').first()
# loading an image 
PL2018 = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia").filterBounds(ee.Geometry.Rectangle([[117.586,0.668],[117.756,0.805]])).filterDate('2018-01-01', '2018-12-31').first()
# loading an image 
PL2017 = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia").filterBounds(ee.Geometry.Rectangle([[117.586,0.668],[117.756,0.805]])).filterDate('2017-01-01', '2017-12-31').first()
# loading an image 
PL2016 = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia").filterBounds(ee.Geometry.Rectangle([[117.586,0.668],[117.756,0.805]])).filterDate('2016-01-01', '2016-12-31').first()
# loading an image 
PL2015 = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia").filterBounds(ee.Geometry.Rectangle([[117.586,0.668],[117.756,0.805]])).filterDate('2015-01-01', '2015-12-31').first()


In [27]:
planet = [PL2015, PL2016, PL2017, PL2018, PL2019, PL2020, PL2021]
study = ee.Geometry.Rectangle([[117.586,0.668],[117.756,0.805]])

In [20]:
pp = pprint.PrettyPrinter(depth=8)

In [21]:
for i in planet:
  pp.pprint(i.getInfo())



{'bands': [{'crs': 'EPSG:3857',
            'crs_transform': [4.777314267159966,
                              0,
                              -20037508.33999997,
                              0,
                              -4.777314267159966,
                              3522218.2628979366],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [8388608, 1413120],
            'id': 'B'},
           {'crs': 'EPSG:3857',
            'crs_transform': [4.777314267159966,
                              0,
                              -20037508.33999997,
                              0,
                              -4.777314267159966,
                              3522218.2628979366],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelTy

In [40]:
# Use folium to visualize the imagery.
map = folium.Map(location=[0.75, 117.7],zoom_start=12) #  note switch between latitude and longitude in folium as opposed to ee.Geometry.Point
#map.setOptions('HYBRID') # To see GE map underneath

for i, y in enumerate(planet):
    map.addLayer(y, {'min': 64, 'max': 5454, 'gamma':1.8, 'bands':['R','G','B']}, f'{i}')
map.addLayer(study,{},'study area')
folium.LayerControl().add_to(map)
map

# Classification - unsupervised example

In [ ]:
def classify(input, training):
  

In [48]:
# making training dataset
input = planet[1]
region = study
training = input.sample(region = region, scale = 4.77, numPixels = 5000)

# initiate and train clusterer
clusterer = ee.Clusterer.wekaKMeans(6).train(training)

# cluster input
result = input.cluster(clusterer)

# display (random colours)
mapB = folium.Map(location=[0.75, 117.7],zoom_start=12) #  note switch between latitude and longitude in folium as opposed to ee.Geometry.Point
mapB.addLayer(input, {'min': 64, 'max': 5454, 'gamma':1.8, 'bands':['R','G','B']}, 'input')
mapB.addLayer(result.randomVisualizer(), {}, 'clusters')
mapB.addLayer(study,{},'study area')
folium.LayerControl().add_to(mapB)
mapB

# how good is the classification?

In [47]:
# compare against Hansen data?